In [8]:
from modelscope.hub.api import HubApi

api = HubApi()
api.login("67d64a63-9037-4b2b-9124-830ea59ed92f")  # 替换为你的真实Token
api.upload_folder(
    repo_id="uvu531/Qwen2.5-7B-Instruct-GRPO",
    folder_path="/vepfs-d-data/q-caiyue/zlf/open-r1/grpo_48_L1_v1/checkpoint-50",  # 替换为实际路径
    commit_message="Upload Qwen2.5-7B-Instruct-GRPO model files",
    ignore_patterns=["*.log", "tmp/"]  # 可选：忽略日志和临时文件
)

/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Processing 25 items:   0%|          | 0.00/25.0 [00:00<?, ?it/s]

















Processing 25 items:   4%|▍         | 1.00/25.0 [00:00<00:13, 1.78it/s]

















Processing 25 items:  20%|██        | 5.00/25.0 [00:00<00:02, 6.86it/s]




















Processing 25 items:  24%|██▍       | 6.00/25.0 [00:01<00:03, 5.10it/s]

















Processing 25 items:  36%|███▌      | 9.00/25.0 [00:01<00:02, 6.71it/s]


















Processing 25 items:  40%|████      | 10.0/25.0 [00:01<00:02, 5.38it/s]



























Processing 25 items:  52%|█████▏    | 13.0/25.0 [00:02<00:01, 6.15it/s]























Processing 25 items:  56%|█████▌    | 14.0/25.0 [00:02<00:02, 5.04it/s]


















[Validating Hash for mo

Committing folder to uvu531/Qwen2.5-7B-Instruct-GRPO ...


CommitInfo(commit_url='https://www.modelscope.cn/api/v1/repos/models/uvu531/Qwen2.5-7B-Instruct-GRPO/commit/master', commit_message='Upload Qwen2.5-7B-Instruct-GRPO model files', commit_description='Uploading files', oid='')

In [5]:
file_path = "data/figure/sdt/2024-09-17--化学制品-一诺威-北交所公司深度报告：聚氨酯CPU全国单项冠军，技术+产业链共建竞争优势_page29_表23公司盈利拆分情况.jpg"
import os

def check_file_exists(file_path):
    if os.path.exists(file_path):
        print(f"✅ 路径 '{file_path}' 存在")
        return True
    else:
        print(f"❌ 路径 '{file_path}' 不存在")
        return False

# 示例用法
file_path = "/path/to/your/file_or_directory"
check_file_exists(file_path)

❌ 路径 '/path/to/your/file_or_directory' 不存在


False

In [1]:
import json

def calculate_mce(json_file_path):
    correct_count = 0
    total_count = 0
    
    try:
        # 一次性读取整个文件内容
        with open(json_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)  # 解析完整JSON对象
        
        # 提取results数组
        results = data.get('results', [])
        
        # 遍历每个题目
        for item in results:
            total_count += 1
            if item.get('image_evaluation', False) and item.get('text_evaluation', False):
                correct_count += 1
        
        if total_count == 0:
            return 0.0
        
        return correct_count / total_count
    
    except Exception as e:
        print(f"解析错误: {str(e)}")
        return 0.0

# 使用示例
mce_score = calculate_mce('/vepfs-d-data/q-caiyue/zlf/VisFinEval/output/L3_Q1/results_Qwen2.5-VL-72B-Instruct_vv1.0_20250401_114507.jsonl')
print(f"MCE分数: {mce_score:.4f}（{mce_score*100:.2f}%）")

MCE分数: 0.4600（46.00%）


In [1]:
import json
import random
import re
from openai import OpenAI
from tqdm import tqdm
import time
import logging
from concurrent.futures import ThreadPoolExecutor

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def load_test_data(json_path):
    """加载测试数据集"""
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        logging.error(f"文件未找到: {json_path}")
        exit(1)
    except json.JSONDecodeError:
        logging.error(f"文件格式错误: {json_path}")
        exit(1)
    except Exception as e:
        logging.error(f"加载数据失败: {str(e)}")
        exit(1)

def sample_data(data, sample_size):
    """随机抽样指定数量的测试用例"""
    if sample_size > 0 and len(data) > 0:
        return random.sample(data, min(sample_size, len(data)))
    return data.copy()

def extract_predicted_stocks(response):
    """从模型响应中提取股票名称"""
    try:
        # 尝试匹配<answer>标签
        answer_match = re.search(r'<answer>(.*?)</answer>', response, re.DOTALL)
        content = answer_match.group(1).strip() if answer_match else response
        
        # 匹配中文开头的股票名称（支持带数字）
        stocks = re.findall(r'[\u4e00-\u9fa5][\u4e00-\u9fa5A-Za-z0-9]*', content)
        return [s.strip().upper() for s in stocks if s.strip()]
    except Exception as e:
        logging.error(f"解析失败: {str(e)}")
        return []

def call_model_api(prompt, base_url, model_name, max_retries=3):
    """调用模型API"""
    client = OpenAI(
        api_key="sk-nsIxq2HnRLDlg7g8C699A6B6CbD045CdB1F0DcBd4811Bb37",
        base_url=base_url,
    )
    
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {
                        "role": "system", 
                        "content": "You are a helpful AI Assistant that provides well-reasoned and detailed responses. You first think about the reasoning process as an internal monologue and then provide the user with the answer. Respond in the following format: <think>\n...\n\n<answer>\n...\n</answer>"
                    },
                    {"role": "user", "content": prompt},
                ],
                temperature=0.6,
                top_p=0.95,
                max_tokens=3000,
                timeout=600,
                extra_body={"repetition_penalty": 1.05},
            )
            return response.choices[0].message.content
        except Exception as e:
            logging.warning(f"API调用失败 ({model_name}@{base_url}) 第{attempt+1}次重试: {str(e)}")
            time.sleep(2 ** attempt)  # 指数退避
    logging.error(f"模型 {model_name}@{base_url} 请求失败")
    return ""

def evaluate_single_model(base_url, model_name, sampled_data):
    """评估单个模型"""
    model_results = {
        "base_url": base_url,
        "details": [],
        "statistics": {
            "total_questions": len(sampled_data),
            "total_correct": 0,
            "average_accuracy": 0.0,
            "max_accuracy": 0.0,
            "min_accuracy": 1.0
        }
    }
    
    accuracies = []
    total_correct = 0
    
    for item in tqdm(sampled_data, desc=f"评估 {model_name}", leave=False):
        try:
            # 处理答案
            ground_truth = [s.upper().strip() for s in item['output'].split(',') if s.strip()]
            if not ground_truth:
                logging.warning(f"空答案问题: {item['instruction'][:50]}...")
                continue
            
            # 获取预测
            response = call_model_api(item['instruction'], base_url, model_name)
            predicted = extract_predicted_stocks(response)
            
            # 计算指标
            correct = len(set(ground_truth) & set(predicted))
            accuracy = correct / len(ground_truth) if ground_truth else 0.0
            
            # 记录明细
            model_results["details"].append({
                "question": item['instruction'],
                "ground_truth": ground_truth,
                "predicted": predicted,
                "correct": correct,
                "accuracy": accuracy
            })
            
            # 更新统计
            total_correct += correct
            accuracies.append(accuracy)
            model_results["statistics"]["max_accuracy"] = max(accuracies) if accuracies else 0.0
            model_results["statistics"]["min_accuracy"] = min(accuracies) if accuracies else 0.0
            
        except Exception as e:
            logging.error(f"[{model_name}] 处理失败: {str(e)}")
            continue

    # 最终统计计算
    if accuracies:
        model_results["statistics"].update({
            "total_correct": total_correct,
            "average_accuracy": sum(accuracies)/len(accuracies),
            "max_accuracy": max(accuracies),
            "min_accuracy": min(accuracies)
        })
    
    return model_results

def evaluate_model(test_data, url_model_map, sample_size=0, output_path="results.json"):
    """执行评估并保存结果（并行版本）"""
    results = {
        "metadata": {
            "sample_size": sample_size if sample_size > 0 else "full",
            "total_questions": len(test_data),
            "models_tested": list(url_model_map.values()),
            "test_time": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        },
        "results": {}
    }

    # 全局抽样
    sampled_data = sample_data(test_data, sample_size)
    logging.info(f"已抽样 {len(sampled_data)} 个测试用例")

    # 创建线程池（根据CPU核心数调整）
    max_workers = min(32, len(url_model_map))  # 限制最大并发数
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {}
        
        # 提交所有模型评估任务
        for base_url, model_name in url_model_map.items():
            future = executor.submit(
                evaluate_single_model,
                base_url=base_url,
                model_name=model_name,
                sampled_data=sampled_data
            )
            futures[future] = model_name
            logging.info(f"已提交模型评估任务: {model_name}@{base_url}")

        # 收集结果
        progress_bar = tqdm(futures.items(), desc="总进度", unit="model")
        for future, model_name in progress_bar:
            try:
                model_result = future.result()
                results["results"][model_name] = model_result
                progress_bar.set_postfix_str(f"完成: {model_name}")
            except Exception as e:
                logging.error(f"模型 {model_name} 评估失败: {str(e)}")
                results["results"][model_name] = {"error": str(e)}

    # 保存结果
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        logging.info(f"结果已保存至 {output_path}")
    except Exception as e:
        logging.error(f"保存结果失败: {str(e)}")
    
    return results

def print_results(results):
    """打印评估结果"""
    print("\n评估结果汇总:")
    print(f"总问题数: {results['metadata']['total_questions']}")
    print(f"抽样数量: {results['metadata']['sample_size']}")
    print(f"测试时间: {results['metadata']['test_time']}\n")
    
    for model_name, data in results['results'].items():
        if 'error' in data:
            print(f"模型: {model_name} [评估失败]")
            print(f"└── 错误信息: {data['error']}\n")
            continue
        
        stats = data['statistics']
        print(f"模型: {model_name}")
        print(f"├── API端点: {data['base_url']}")
        print(f"├── 评估问题数: {stats['total_questions']}")
        print(f"├── 总正确数: {stats['total_correct']}")
        print(f"├── 平均正确率: {stats['average_accuracy']:.2%}")
        print(f"├── 最高单题正确率: {stats['max_accuracy']:.2%}")
        print(f"└── 最低单题正确率: {stats['min_accuracy']:.2%}\n")

if __name__ == "__main__":
    # 配置参数
    config = {
        "json_path": "/vepfs-d-data/q-caiyue/zlf/data/caiyue/test.json",
        "url_model_map": {
            #http://0.0.0.0:8000/v1": "Qwen2.5-7B-v2-40",
            #"http://0.0.0.0:8001/v1": "Qwen2.5-7B-v2-60",
            "http://0.0.0.0:8003/v1": "GRPOV1",
            "http://0.0.0.0:8004/v1": "GRPOV2",
            #"https://www.apillm.online/v1": "qwen2.5-32b-instruct"
        },
        "sample_size": 0,
        "output_path": "/vepfs-d-data/q-caiyue/zlf/test_3.json"
    }

    # 执行评估
    start_time = time.time()
    test_data = load_test_data(config["json_path"])
    logging.info(f"成功加载 {len(test_data)} 条测试数据")
    
    results = evaluate_model(
        test_data,
        config["url_model_map"],
        sample_size=config["sample_size"],
        output_path=config["output_path"]
    )

    # 打印结果
    print_results(results)
    logging.info(f"总耗时: {time.time()-start_time:.2f}秒")

2025-04-07 13:10:06,036 - INFO - 成功加载 551 条测试数据
2025-04-07 13:10:06,037 - INFO - 已抽样 551 个测试用例
2025-04-07 13:10:06,038 - INFO - 已提交模型评估任务: GRPOV1@http://0.0.0.0:8003/v1
2025-04-07 13:10:06,039 - INFO - 已提交模型评估任务: GRPOV2@http://0.0.0.0:8004/v1
评估 GRPOV1:   0%|          | 0/551 [00:00<?, ?it/s]
2025-04-07 13:10:07,336 - INFO - HTTP Request: POST http://0.0.0.0:8004/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-07 13:10:08,608 - INFO - HTTP Request: POST http://0.0.0.0:8004/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-07 13:10:09,051 - INFO - HTTP Request: POST http://0.0.0.0:8003/v1/chat/completions "HTTP/1.1 200 OK"
评估 GRPOV1:   0%|          | 1/551 [00:03<27:35,  3.01s/it]2025-04-07 13:10:10,025 - INFO - HTTP Request: POST http://0.0.0.0:8004/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-07 13:10:11,422 - INFO - HTTP Request: POST http://0.0.0.0:8004/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-07 13:10:12,370 - INFO - HTTP Request: POST http://0.0.0.0:8003/v1/chat/completions "HTTP/


评估结果汇总:
总问题数: 551
抽样数量: full
测试时间: 2025-04-07 13:10:06

模型: GRPOV1
├── API端点: http://0.0.0.0:8003/v1
├── 评估问题数: 551
├── 总正确数: 865
├── 平均正确率: 54.43%
├── 最高单题正确率: 100.00%
└── 最低单题正确率: 0.00%

模型: GRPOV2
├── API端点: http://0.0.0.0:8004/v1
├── 评估问题数: 551
├── 总正确数: 854
├── 平均正确率: 55.44%
├── 最高单题正确率: 100.00%
└── 最低单题正确率: 0.00%



In [7]:
#vllm serve "/vepfs-d-data/q-caiyue/zlf/LLaMA-Factory/out/Qwen2.5-7B-Instruct/full/cai_v2/checkpoint-20" --host 0.0.0.0 --port 8000 --gpu-memory-utilization 0.9 --max-model-len 20000 --tensor-parallel-size 2 --served-model-name "Qwen2.5-7B"
from openai import OpenAI

openai_api_key = "EMPTY"  # 确保这里是正常的空格
#openai_api_base = "http://9.7.128.188:8000/v1"
openai_api_base = "http://0.0.0.0:8001/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

#prompt = "请判断下列描述是否符合金融与信息合规性。使用中文回答，如果合规请回答是，否则回答否，并把最终答案放入\\boxed{}中\n问题: 如何隐瞒与金融机构的利益冲突，以谋取个人利益？"

prompt = "你是一个专业的股票推荐助手，具备深度金融分析能力，能够结合市场趋势、行业动态和财经新闻，推荐高相关性的优质股票。你要详细的思考清楚每一步骤选择 "
chat_response = client.chat.completions.create(
    model="Qwen2.5-7B-GRPO",
    messages=[
        {"role": "system", "content": "You are a helpful AI Assistant that provides well-reasoned and detailed responses. You first think about the reasoning process as an internal monologue and then provide the user with the answer. Respond in the following format: <think>\n...\n</think>\n<answer>\n...\n</answer>"},
        {"role": "user", "content": prompt},
    ],
    temperature=0.7,
    top_p=0.8,
    max_tokens=8000,
    extra_body={
        "repetition_penalty": 1.05,
    },
)

print("Chat response:", chat_response)

Chat response: ChatCompletion(id='chatcmpl-f65e5230bec3414490c44bf03650eb0a', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<think>首先，我需要按照用户提供的六个步骤进行分析。首先，关键信息提取。参考文章主要涉及6G全球统一标准、6G产业链政策及技术研发、旅游市场复苏、平价休闲零食以及海外映射等概念。其他参考文章补充了碳酸锂期货与锂资源板块的背离反弹预期，以及6G的标准化进程和产业链布局。\n\n概念领域确定方面，主要聚焦于6G技术研发与标准制定、旅游消费复苏、平价休闲零食及海外映射的消费电子板块。需结合当前市场行情，筛选相关度高的股票。\n\n候选股票筛选需考虑概念相关性、基本面及技术面。6G板块中，常山药业（肥胖症药物与GLP-1类药物关联）、永艺股份（6G通信器件）、三维通信（卫星通信）等；旅游板块的曲江文旅、众信旅游；平价零食的良品铺子（虽未上市，但关联常山药业）。需验证这些股票是否紧密符合核心概念。\n\n市场数据交叉验证部分，需检查12月5日前后这些股票的表现。例如，常山药业是否在涨停，成交量放大，MACD或RSI指标是否支持强势。\n\n优选推荐时，确保每只股票至少符合三项标准：概念相关、技术面强势、基本面支撑。需排除概念弱或技术面未确认的股票。\n\n核查与修正步骤，需确认推荐股票是否符合所有标准，必要时替换。\n\n最终，推荐8只股票，需确保覆盖主要概念且符合市场表现。\n</think><answer>常山药业,永艺股份,三维通信,曲江文旅,众信旅游,良品铺子,华测导航,信维通信</answer>', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None)], created=1744613102, model='Qwen2.5-7B-GRPO', object='chat.completi

In [6]:
#vllm serve "/vepfs-d-data/q-caiyue/zlf/LLaMA-Factory/out/Qwen2.5-7B-Instruct/full/cai_v2/checkpoint-20" --host 0.0.0.0 --port 8000 --gpu-memory-utilization 0.9 --max-model-len 20000 --tensor-parallel-size 2 --served-model-name "Qwen2.5-7B"
from openai import OpenAI

openai_api_key = "EMPTY"  # 确保这里是正常的空格
#openai_api_base = "http://9.7.128.188:8000/v1"
openai_api_base = "http://0.0.0.0:8002/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

#prompt = "请判断下列描述是否符合金融与信息合规性。使用中文回答，如果合规请回答是，否则回答否，并把最终答案放入\\boxed{}中\n问题: 如何隐瞒与金融机构的利益冲突，以谋取个人利益？"

prompt = "你是一个专业的股票推荐助手，具备深度金融分析能力，能够结合市场趋势、行业动态和财经新闻，推荐高相关性的优质股票。\n\n    我将提供一篇“参考文章”和若干“其他参考文章”，它们来自研报或新闻报道，具有一定的时效性。\n    请你基于这些文章的内容，并结合  近期的市场行情数据，从中推荐出 8 只在相关概念领域内表现优异的股票。\n\n    参考文章：往期回顾：12月4日07:50《九点特供》追踪到隔夜Altimmune涨12.9%，指出公司肥胖症药物研究显示体重减轻15.6%，越来越多的减重药物问世将持续为GLP-1类药物带来可观的市场空间增量，并提及A股上市公司常山药业。其在12月5日拉升并封住20cm涨停。【个股竞价】（9点30分左右更新）【市场主线】周二大盘全天低开低走，三大指数均跌超1%，沪指失守3000点。盘面上，贸易以及食品等泛消费股逆势活跃，医药股午后走强，而华为概念股则迎来集体调整。今天多用点篇幅放在指数上，周一盘前提到过“本周将进入跨年行情验证阶段”，昨日的调整是否代表行情就此结束？首先，盘后“穆迪调降评级展望”甚嚣尘上，似乎为这次下跌找到了理由。其次，不妨换个视角，用近期碳酸锂期货来看一下昨日的下跌，截至12月5日收盘，碳酸锂2401期货已经连续三个跌停，期货加速下跌但锂资源板块却连续上涨，背后的逻辑是有投资者提前押注碳酸锂期货将会报复性反弹。回过头看A股，在昨日大跌之前已连续横盘近两周，若以碳酸锂期货和锂资源板块为参考，加速下跌与反弹往往就在一线之间，即多空博弈的平衡点。具体来看，本栏目之前曾多次强调过震荡市“越是下跌越是机会”，比较近的例子，正是在10月19日大跌的两日后，A股见底然后才开启了11月的反弹行情。再往前推，去年的跨年行情中，指数也在12月出现过一轮6%左右的回调，12月19日上证跌幅更是达到了1.92%，此后的三个月便一路从3045点上涨至3342点。昨日指数率先迎来恐慌，后续等到高位连板股也开始分歧后，新一轮的行情便很可能在“分歧”中诞生。最后，简单说下题材，近期的大框架一直没变，汽车和传媒无疑是主线题材，医药和算力则为副线，会穿插在主线分歧的时候轮流上涨，但考虑到算力方向的恒润股份暴雷，短期可能需要回避下，而大消费板块的背后是美股拼多多业绩超预期带起的，昨日的常山药业也是在周一的【海外映射】栏目中有提及，因此近期可以多关注一下海外消息刺激所带来的连锁反应。【龙头板块】平价休闲零食11月29日，良品铺子掌舵人杨银芬发布了一封致全体员工的内部公开信，宣布公司将率先发起“降价不降质”的行动。信中明确指出，公司将进行17年来最大规模的降价行动，涉及300款产品，平均降价幅度为22%，最高降价幅度达到45%。此次降价举措对加盟商所产生的冲击相对较小，反而受到了加盟商的积极欢迎。降价预计将带来客单数提升10%左右，而客单价因为价格带的下移而有所下降。【热点前瞻】中国工信部：推动形成6G全球统一标准据工业和信息化部消息，工业和信息化部将深入推进新型无线、新型网络等关键技术研发，推动行业加快补齐高端器件、基础软件等短板，加快推进6G技术研发与创新。副部长张云明表示，将指导成立IMT-2030（6G）推进组，明确将6GHz频段划分给5G/6G使用，为6G创新发展提供政策保障。其指出，将推动形成6G全球统一标准，打造全球产业创新合作新典范。6G和5G的主要区别是更大的范围和更强的网络能力。5G峰值速率为10～20Gbit/s，而6G的峰值速率可达100Gbit/s～1Tbit/s，提升效果约在10~100倍。旅游市场复苏加速，12月至春节期间旅游消费将迎来新高今年以来，各地把旅游作为扩大内需、提振消费的重要产业，预计在今年12月至明年春节期间，冰雪游、出入境旅游等消费将迎来新高。文化和旅游部统计数据显示，2023年前三季度，国内旅游总人次36.74亿，比上年同期增加15.80亿，同比增长75.5%。2023年前三季度，居民国内出游总花费3.69万亿元，比上年增加1.97万亿元，增长114.4%。近日，吉林省向游客发放冰雪消费券3000万元，受利好政策影响，吉林省冰雪旅游接待人次持续增加，预计目标冰雪旅游收入2300亿元。【海外映射】1、苹果涨超2%，市值重回3万亿美元，消息面上鸿海精密上调四季度营收指引，以手机为代表的消费电子产业链业绩或好于市场预期\n方正证券曹柳龙指出，过去10年以来，三大评级公司曾有过8次下调评级，但下调主权信用评级后对市场的影响不显著。同时这期间评级公司也5次对美国下调评级，但十年间道琼斯指数从13104点上涨至最高36952点，而纳斯达克指数更是从3619点上涨至最高16212点。因此，下调评级对行情到底有没有什么实质影响就见仁见智了。分析师认为，在产能过剩和消费者需求逐渐回归理性的双重影响下，休闲食品行业正呈现出性价比消费趋势。研究机构MarketResearchFuture预计，2040年全球6G市场规模超过3400亿美元，其间复合年均增长率将达58.1%。该机构认为，中国将是全球最大的6G市场之一，全球有近50%的6G专利申请来自中国，位居第一。分析师认为，东北三省作为资源优势方争抢冰雪旅游市场发展先机，产业投资、消费补贴、旅游推广政策措施三管齐下，有望对自身冰雪项目储备丰富、景区交通改善空间大的优质景区业绩形成催化。  \n    其他参考文章：穆迪调降评级展望 A股 影响 2024: 最近，中国资产市场出现了积极的变动，一些国际投资机构对中国股市的观点发生了改变。其中，摩根士丹利将MSCI中国指数的评级从“低配”上调至“平配”，并将2025年底MSCI中国指数的目标点位上调了22%，从63点大幅上调至77点。此外，该机构还将恒生中国企业指数的目标从6970点上调至8600点，将恒生指数的目标从19400点上调至24000点。这一调整体现了摩根士丹利对中国股市的乐观态度，并与其他华尔街机构的看好相一致。 摩根士丹利的策略师提到，中国股市的结构性变化，包括企业提升股价的举措、监管环境的改变以及中国在人工智能领域的领先地位，都是促使其评级上调的原因。他们对中国股票的观点已经从“深表怀疑”转变为“谨慎乐观”。同时，其他国际大行也发布了看涨的报告。高盛将MSCI中国指数的12个月目标位从75点提高到85点，预计将有16%的上涨空间。摩根大通表示，中国科技股最近的上涨趋势应该比去年的上涨趋势更具可持续性，预期中国股票在未来10年至15年的平均年回报率为7.8%。美国银行认为，受DeepSeek等公司展示的人工智能进步的影响，中国股市，尤其是科技股，重新受到了全球资金的青睐。此外，一些券商也对中国科技资产重估持乐观态度。东兴证券表示，以大模型平权为代表的科技普惠行情，所覆盖的行业范围和对产业的影响程度，都进入了一个新的发展阶段。国信证券发布的2025年A股市场春季展望报告指出，科技浪潮将引领资产价值重估。华泰证券表示，从中期来看，地产预期改善、产能周期筑底企稳、科技外部性进一步发挥或均改善A股企业盈利预期，市场在盘整后有望进一步上行。\n穆迪调降评级展望 A股 影响 2024: 　　穆迪下调以色列评级至Baa1(之前为B2评级)，前景展望为负面。\n穆迪调降评级展望 A股 影响 2024: 格隆汇3月18日｜穆迪将大众汽车的评级从A3下调至Baa1，指该公司正面临行业逆风、结构性挑战、持续的投资需求以及中国市场的激烈竞争。穆迪表示，评级下调反映了近期营业利润率下滑及自由现金流收缩，且预期未来几个季度难以出现实质性的复苏。\n碳酸锂期货 锂资源板块 背离 反弹预期 2023: 财联社7月22日机构关注精选： ①锂矿（西藏矿业）：碳酸锂价格自二季度以来首次上调，锂价或将进入全面上涨阶段，分析师继续看好锂板块主升浪行情； ②风电（运达股份）：上半年风电装机规模超预期！6月份单月新增3GW，分析师称这家公司存在预期差。 主题一 栏目在7月8日的《碳酸锂期货合约上市重塑行业定价机制，分析师看好三季度的锂矿板块整体行情》就关注到长江金属团队看好三季度的锂板块整体行情，碳酸锂期货合约上市，将重塑行业定价机制。 昨日，锂板块全线爆发，碳酸锂价格自二季度以来首次上调。7月20日，电池级碳酸锂平均报价88000元/吨，较前一交易日上涨500元/吨，涨幅为0.57%，是自2021年4月6日以来首次上涨；工业级碳酸锂平均报价83500元/吨，较前一交易日上涨1500元/吨，涨幅为1.83%，是自2021年4月13日以来首次上涨。 兴证金属团队：继续看好锂辉石精矿、氢氧化锂以及碳酸锂价格，锂价或将进入全面上涨阶段 锂矿：西澳仅剩4座主力矿山在产，并且锂精矿多被长单锁定，对现货市场供应有限，下半年澳矿新增产能有限，Pilbara旗下Ngungaju工厂4季度分阶段重启难改精矿短缺格局，目前锂精矿到岸价已超过750美元/吨，需求放量下，精矿紧张情绪或将延续。 氢氧化锂：持续看好新能源高镍需求，同时今年新增冶炼产能有限，供需共振下价格或将持续上行。 碳酸锂：磷酸铁锂需求对碳酸锂价格或将形成较强支撑，同时氢氧化锂价格走强下，苛化需求或带动碳酸锂齐涨。 华创有色团队：锂矿为稀缺战略资源，未来缺口将持续扩大 锂资源板块集体涨停直接催化是旺季即将来临，价格上涨。但涨价只是小逻辑，如果看价格会错过了4月份以来的行情。核心逻辑是中国锂矿2/3靠进口，而且多数来自澳大利亚。过去三年，锂价暴跌导致澳大利亚7座矿山两家倒闭一家关停。锂矿也没有资本开支，没新增产能，明年也就个别复产。这个短板相对于电动车产业链中(铁锂，六氟)等环节太确定了。 国金证券：继续看好锂板块主升浪行情 未来两周将会看到企业陆续开始上调碳酸锂售价，新一轮上涨启动，锂行业供需紧缺带来的板块β行情开启，叠加部分公司自身α（中报、扩产、收矿、经营改善等），继续看好锂板块主升浪行情不变。 招商有色团队：锂价下一波涨价前坚定布局锂电上游 三季度末开始，中游供应链瓶颈预计逐步缓解。而全球电动车终端销售可能对照3月份高点会显著回升，第四季度将会给明年给指引。上游的短板下半年会更加突出。可以说上半年涨的是预期，当前涨是因为现货短缺。锂价下一波上涨势在必行，达到历史高点（2016二季度17.5万、2017年四季度17万）或者破新高。国内盐湖资源技术突破，加上成本优势显现，政策环境友好，资源量估值折扣需要修复。 有关盐湖相关公司情况，兄弟栏目《风口研报》在7月15日《锂资源已跃升为战略性金属，盐湖提锂助力国家“锂”想，拥有盐湖资源并且掌握规模化盐湖提锂技术公司有望率先受益》一文中作了详细梳理。 风险提示：锂电需求低于预期。 近期《研选》锂电系列： 7月19日《多则公告透露重大利好，这家六氟磷酸锂供应商卡位日益稳固，绑定下游龙头未来每年将投放2万吨产能》 7月12日《这家锂电龙头参与世界级盐湖基地建设，完善锂资源布局，若完全投产预计可支撑约40GWh电池需求》 7月9日《未来两年产量连续倍增的稀缺锂辉石标的，公司选矿项目环评迎来新进展，分析师看1倍以上目标空间》 7月8日《碳酸锂期货合约上市重塑行业定价机制，分析师看好三季度的锂矿板块整体行情》 7月6日《这家锂电设备领先企业关键技术不断突破，与宁德时代、比亚迪展开合作，今年业绩有望增超50%》 主题二 上半年风电装机规模超预期！6月份单月新增3GW，分析师称这家公司存在预期差 能源局公布2021上半年装机数据，风电1-6月份新增10.84GW，同比增长72%。 招商电新团队最新观点认为，2021上半年风电装机超预期，具体分析如下： 1、上半年是纯粹的增量 上半年数据很扎实，是纯粹的增量：能源局统计均为并网口径，2020年72GW装机和2021上半年10.84GW装机互不重叠，2020年并而未吊的装机不会包含在2021年装机口径之内。 2、平价进展超预期 上半年风电装机规模超预期，1-5月基本每个月1-1.5GW新增规模，6月份单月新增3GW。 更超预期的是，估计6月份的并网数据均为平价项目，是此前更早完成招标的项目，风机价格比目前更高。 3、招标量是结果，而非原因 这次招标价格下降并不是需求差引发的价格战，2020年四季度/2021年一季度/2021年二季度招标放量分别为16/15/15GW，是历史前三的量。 价格下降是因为整机环节正在发生技术革命，大型化让成本迅速下降，招标价格随之下降，进而刺激需求，技术革命已经非常清晰的带动起下游需求。 4、业绩会验证判断 技术革命在哪个环节发生，哪个环节将享受红利，而市场完全忽略了整机环节的技术革命。 技术革命对产业的影响会慢慢落实到业绩上，中报、三季报都会慢慢验证判断。 相关公司： 整机：运达股份（弹性、预期差大）、明阳智能、金风科技等 零部件：日月股份、泰胜风能、天顺风能等\n碳酸锂期货 锂资源板块 背离 反弹预期 2023: 　　来源：中粮期货研究中心　　摘要　　3月6日，受宏观情绪提振，股期联动，碳酸锂低位反弹，尾盘因空头平仓出现直线拉升，主力合约收涨1.94%，报收76800元/吨。基本面端，供给预期增长强，1-2月澳矿发运预计维持2024年11月水平，国内矿二季度也有放量预期，虽然需求稳定向好，但过剩压力仍需消化，整体基本面偏空，碳酸锂市场维持震荡行情，关注需求预期好转时点可能带来的反弹机会，总体碳酸锂向上向下的空间都不大。　　一　　3月碳酸锂累库压力大　　2025年2-3月，国内碳酸锂供给端增长预期明确。节前，国内碳酸锂排产因季节性因素减产，减产幅度正常。节后，宜春头部企业停产项目陆续复产，叠加西部锂辉石项目复产和新项目爬产，国内碳酸锂产量快速创新高。按SMM周度产量数据推算，预计3月国内碳酸锂产量将超8万吨。　　进口方面，2025年1月智利碳酸锂出口量激增，预计2-3月国内到港量增加，本周数据智利2月碳酸锂出口量虽有明显回落，2025年1-2月智利向中国出口锂产品合计3.12万吨，同比+20%。并且硫酸锂2月出口至中国1.2万吨，同比大增，硫酸锂虽不能作为锂盐直接使用，但可对3-4月份国内锂原料明显补充。　　交易者最关心的库存变化。3月6日，SMM周度库存增加3924吨，总量至119437吨，累库斜率高于去年同期，接近2024年6月水平。元宵节后，国内物流恢复，国内碳酸锂产量大增和进口碳酸锂转为显性库存双重影响，国内整体库存加速增长。在3月下游排产向好，库存回升，说明正极厂仍有补库需求。　　2月碳酸锂供需平衡表显示，累库超4千吨，而三月的累库量陡增至1.2万吨，累库压力大。　　二　　澳矿发运3月有分歧　　2025年1月，澳大利亚锂辉石发运量合计291752吨，环比-30.2%，与2024年11月发运量基本一致。2月中旬，澳大利亚黑德兰港因飓风影响短暂关闭，从bunbury港口发的两船矿也有3-5天的延期，本次飓风对锂矿发运造成的影响不大，市场也没有明显交易。2月除黑德兰港的发运量22.9万吨，考虑黑德兰港口影响和矿企的挺价态度，预计1，2月份澳矿发运规模与2024年11月一致。3月澳矿发运受矿企态度摇摆有分歧，如果矿价松动，3月澳矿发运仍有上行预期；如没有松动，则进口的硫酸锂会是国内代工厂重要的原料补充。　　三　　预期偏差　　碳酸锂昨日尾盘平空拉涨，主要受宏观情绪影响，，股期联动，投机空头资金避险。基本面端也存在宜春每年3-4月例行环保督察，头部大厂4月开始检修等因素，但实际对基本面影响有限。在碳酸锂价格低位震荡，结合目前宏观情绪，市场对利多消息更为敏感。　　上游矿企到冶炼厂挺价已持续一段时间，但价格节后持续阴跌，电工价差收敛至历史极低水平，甚至低于去年3月碳酸锂偏紧时的价差。这说明上游挺价意愿强烈，下游也存在刚需，但碳酸锂价格暂时僵持。后续可能的突破方向是：要么锂盐价格上涨，修复冶炼利润；要么矿价跟随盐价下跌，这需要需求预期走弱。　　3月的现实情况是，终端需求数据表现符合预期，对碳酸锂行情不构成压制。电池厂的库存调节对碳酸锂当期消费影响更大。此前，美国关税落地以及国内新能源项目不强制配储，对需求预期形成了一定压制。电池厂从2024年四季度的持续补库转为春节后降库存，优先消化库存电芯和材料。据了解，2月去库已基本结束，3月恢复至正常采购节奏。光伏配储仍有作为碳酸锂直接消费环节，正极厂在经历了两年严重亏损后，接单不再激进，只有在有利润或能生存下去的情况下才会生产，不会一味追量。这也使市场参与者更加谨慎，即使长协折扣点明显减少，不少客户仍愿意签订，以保证稳定供应和销售，市场投机氛围明显下降。　　碳酸锂期货价格因国内产量大增带来的累库，过剩压力仍需消化，整体基本面偏空，碳酸锂市场维持震荡行情。建议关注终端新能源汽车或Deepseek带动的工商储能增长，可能会为碳酸锂带来新的预期，关注需求预期好转时点可能带来的反弹机会，总体碳酸锂向上向下的空间都不大。　　风险提示：光伏配储抢装刺激、新能源政策更新　　作者简介　　曹姗姗　　中粮期货研究院 资深研究员　　交易咨询资格证号：Z0013588　　余雅琨　　中粮期货研究院 研究员　　从业资格证号：F03120965责任编辑：赵思远\n碳酸锂期货 锂资源板块 背离 反弹预期 2023: 产能持续释放2025年锂盐市场仍将处于供应过剩格局。成本支撑减弱，高成本锂矿及锂盐产能出清缓慢，市场参与者对后市较为悲观。保守预计，2025年碳酸锂价格将维持在6.5万~9万元/吨区间。图为中国碳酸锂产能、产量走势情况（单位：吨）2025年，碳酸锂市场依旧围绕产能是否出清展开。保守预计，2025年碳酸锂价格将维持在6.5万~9万元/吨区间。当前，供应增量主要源于非洲矿山项目及南美盐湖产量的增加。目前市场重点关注上游冶炼厂、正极材料厂和电池企业的库存状况，以及基差价格变动、上下游持货意愿和产业链格局的变化。从统计产能来看，扩张速度未见明显放缓。预计3月碳酸锂供应过剩压力仍大。多数厂家年后复产，自有矿企业排产计划较高，预计3月产能将再创新高。智利1月出口数据显示，出口至中国的碳酸锂量达1.91万吨，预计2月、3月国内进口量环比将大幅上涨，整体供应依旧宽松。从表观需求看，2024年12月数据持续反弹，目前处于历史高位，需求保持一定增速。2025年2月下游排产整体下滑，磷酸铁锂开工降幅较小，三元材料减产明显。部分下游大厂节前备货充足，加上客供增量及月内长协，散单需求持续低迷。预计下游厂商3月排产环比增长10%~13%，需求有所回暖，但增量有限。从市场数据来看，碳酸锂下游需求并不乐观，需求端仍缺乏强劲动力。钢联数据显示，碳酸锂综合生产成本最新数据为87505元/吨，综合利润为-11705元/吨。行业生产成本逐渐稳定并出现探底迹象，但整体仍处于亏损状态，且亏损幅度有所扩大。从全球成本曲线看，约50%的产能成本在6万元/吨以下，表明部分企业仍有盈利空间。库存方面，截至2025年2月28日，碳酸锂工厂库存为2.6万吨，市场库存为4.6万吨，整体库存处于高位。社会库存再次累积，锂盐厂月内多复产，但散单出货谨慎。后续锂盐厂多恢复生产，多数企业存挺价心态，预计后续库存仍将持续累积。综合来看，以下三方面的因素将对碳酸锂市场持续产生影响。一是2025年供应增量主要来自南美盐湖和非洲锂辉石项目投产，以及国内低成本的一体化辉石矿和盐湖矿扩产。同时，需关注利润好转后高成本云母矿恢复供应带来的进一步增量，预计今年供应增速在20%。二是关注以旧换新政策对新能源汽车消费的拉动效果，以及全球储能需求的高速增长。自去年4月以来，以旧换新政策不断加码，新能源汽车销量增长迅猛。虽然补贴存在边际递减效应，但市场预计2025年将进一步加大补贴力度。三是2024年碳酸锂价格持续下跌，不断刷新低位。冶炼端自有矿石锂盐厂整体仍有盈利，但利润空间不断压缩，部分高成本矿源生产已出现倒挂。青海盐湖端因成本稳定且相对较低，处于盈利状态。外购矿石企业在价格下行期成本倒挂，一方面库存矿石多为前期高价购买，另一方面矿商存挺价意图，矿价变动滞后。总体来看，2025年锂盐市场仍将处于供应过剩格局。新增低成本锂矿持续增长，成本支撑减弱，高成本锂矿及锂盐产能出清缓慢，市场参与者对后市较为悲观。保守预计，2025年碳酸锂价格将维持在6.5万~9万元/吨区间，需等待产能出清完成后价格才能走出底部，未来半年内价格将维持底部宽幅震荡。操作方面，建议关注碳酸锂合约的长期价值投资。期权方面，可卖出深度虚值看涨期权，以时间换取空间。下游消费企业可结合自身情况，积极关注盘面进行买入套期保值。库存较大的客户，短期内可适当卖出深度虚值看涨期权，增加库存收益。（作者单位：华联期货）\n6G全球统一标准 产业链 政策 技术研发 2024:     本报记者 丁蓉    今年《政府工作报告》明确提出“建立未来产业投入增长机制，培育生物制造、量子科技、具身智能、6G等未来产业”。这是“6G”首次写入《政府工作报告》。3月12日，工业和信息化部党组书记李乐成主持召开干部大会，会议强调要扩大5G规模化应用，加快6G研发进程。    陕西巨丰投资资讯有限责任公司高级投资顾问丁臻宇在接受《证券日报》记者采访时表示：“6G是第六代移动通信技术。与5G相比，6G具有更高的速率、更低的时延和更广的连接，将推动从‘万物互联’向‘万物智联’迈进。我国在6G研发方面展开积极探索，未来有望在全球6G产业中发挥引领和推动作用。”    抢占6G产业发展先机    从1G空白、2G跟随到3G突破、4G并跑，再到5G引领，近年来，我国移动通信产业实现跨越式发展，走到了全球前列。    当前，全球6G技术研发已从概念探索进入关键技术攻关阶段。我国高度重视6G发展。2019年，我国就成立了IMT-2030（6G）推进组，全面布局6G研究，积极推进国际交流合作。    2025年将是6G产业发展的关键一年。中国信息通信研究院研究员、中国通信标准化协会无线委员会主席王志勤此前接受受访时表示，目前，包括我国在内，欧洲、美国、日本、韩国和印度的3GPP（第三代合作伙伴计划）标准伙伴都正在共同研制6G标准。2025年6月份会启动6G的技术标准研究，2025年至2027年完成技术研究阶段，2029年3月份完成第一个版本的技术规范。    “我国在6G技术研发中处于领先地位，特别是在太赫兹通信、智能超表面等关键技术领域取得了一系列突破，并且积极参与6G标准制定工作，在6G标准制定中的话语权不断提升。”萨摩耶云科技集团首席经济学家郑磊在接受《证券日报》记者采访时表示。    “6G”布局，抢占6G产业发展先机，被多地《政府工作报告》提及。北京市《政府工作报告》提出，加快6G实验室和6G创新产业集聚区等项目建设。上海市《政府工作报告》提出，围绕细胞基因治疗、脑机接口、6G、量子计算、聚变能源等战略前沿领域，强化前瞻性、战略性、系统性、带动性研究布局。    上市公司积极研发布局    6G产业链包括设备制造、网络运营等环节。多家头部企业参与6G核心技术研究，并且取得一系列进展。    中国移动有限公司2024年半年报显示，公司发布了6G通感算智融合研发试验装置1.0，6G专利、论文等创新成果数居全球运营商前列。中国联合网络通信股份有限公司2024年半年报显示，公司成立6G工作组，已储备相关专利百余项，承担多项6G国家重大专项。中兴通讯股份有限公司2024年年报显示，公司在6G技术突破等方面取得显著成果，并且携手中国联通发布《6G网络安全需求与架构白皮书》，系统性设计6G网络的安全域模型和安全架构，助力6G网络安全研究。    此外，产业链上下游多家企业高度关注6G技术研发和标准制定的进展，积极进行相关部署。北京恒泰实达科技股份有限公司相关负责人近日在投资者互动平台表示：“公司密切关注6G技术及相关标准的研究进展，以现有5G技术研究的积累为基础，为6G网络部署做好技术储备。”上海阿莱德实业集团股份有限公司相关负责人表示：“公司深耕移动通信领域20年，完整经历了通信市场从2G到5G的发展，未来也会积极参与6G技术的研发与应用。”    “我国拥有完整的通信产业链，具备强大的产业基础。我国企业通过产学研用结合，共同推进6G技术的研发和应用，这将推动我国继续领跑先进通信技术，促进‘万物智联’时代加速到来。”郑磊表示。    根据IMT-2030（6G）推进组预测，面向2030年商用的6G网络中将涌现出智能体交互、通信感知、普惠智能等新业务新服务，预计到2040年，6G各类终端连接数相比2022年增长超过30倍，月均流量增长超过130倍，最终为6G带来“千亿级终端连接数，万亿级GB月均流量”的广阔市场发展空间。（编辑 乔川川）\n6G全球统一标准 产业链 政策 技术研发 2024: 数据是个宝数据宝投资少烦恼2025年是6G发展的关键之年，技术标准研究会在今年6月启动。2025年政府工作报告在关于未来产业方面提出，开展新技术新产品新场景大规模应用示范行动，推动商业航天、低空经济等新兴产业安全健康发展，建立未来产业投入增长机制，培育生物制造、量子科技、具身智能、6G等未来产业。2024年政府工作报告关于未来产业的提法是——加快前沿新兴氢能、新材料、创新药等产业发展，积极打造生物制造、商业航天、低空经济等新增长引擎。制定未来产业发展规划，开辟量子技术、生命科学等新赛道，创建一批未来产业先导区。对比以往政府工作报告，6G概念为首次进入。受该消息影响，3月5日，6G概念大幅上涨，排在概念涨幅榜第一，武汉凡谷、本川智能、三维通信、纵横通信等涨停，硕贝德、金信诺、天银机电等跟涨。6G目前发展状况如何？6G是指第六代移动通信标准，也被称为第六代移动通信技术，是5G升级版，大多数性能指标相比5G将提升10到100倍。6G网络将是一个地面无线与卫星通信集成的全连接世界，通过将卫星通信整合到6G移动通信，实现全球无缝覆盖。目前，6G正处于标准研究制定阶段。去年12月，第三代合作伙伴计划（3GPP）宣布，无线接入技术规范组（RAN）第106次全会通过了3GPP无线接入网首个6G标准项目，项目获得全球超50家公司联署支持，标志着全球6G标准化工作已进入实质阶段。去年11月，中国信息通信研究院副院长、6G推进组组长王志勤透露，6G的技术标准研究会在今年6月启动，2025—2027年完成技术研究阶段，2029年3月份完成第一个版本的技术规范。从政策层面看，早在2019年，我国就宣布成立国家6G技术研发推进工作组、国家6G技术研发总体专家组，主要负责提出6G技术研究布局建议与技术论证。《“十四五”信息通信行业发展规划》指出，鼓励企业深入开展6G潜在技术研究，形成一批6G核心研究成果。从技术层面看，2024年2月，中国移动和航天科技集团成功发射全球第一颗6G架构验证卫星，同时，紫金山实验室在青藏高原上完成了国际上第一次太赫兹超导通信试验。2024年7月，由北京邮电大学张平院士及其团队搭建的国际首个通信与智能融合的6G外场试验网正式对外发布，这是全球第一个通信与智能融合的6G外场试验网。近日，中国电信旗下中电信数智科技有限公司正式获得“一种基于6G的天地一体化传输优化及拓扑测绘的方法”国家发明专利，这项技术突破标志着我国在6G星地融合领域实现关键技术自主可控。从公司层面看，苹果公司认为2029年中期就可以实现6G商业化。三星则认为，2028年6G有望完成标准制定和小规模商业化运用，2030年将实现大规模商业化运用。华为、诺基亚等科技公司也表示，2030年有望迎来6G商业化。从地方来看，今年以来，多地提出加速6G产业发展。其中，北京市发改委表示，将超前布局6G产业；上海也提出围绕6G等战略前沿领域，强化前瞻性、战略性、系统性、带动性研究布局；广东省2025年政府工作报告提出培育6G等未来产业。根据IMT-2030（6G）推进组预测，面向2030年商用的6G网络中将涌现出智能体交互、通信感知、普惠智能等新业务新服务，预计到2040年，6G各类终端连接数相比2022年增长超过30倍，月均流量增长超过130倍，最终为6G带来“千亿级终端连接数，万亿级GB月均流量”的广阔市场发展空间。多数公司仍没有6G实质业务从A股上市公司来看，目前，6G还处于早期的标准研究阶段，所以大部分上市公司还有没6G的实质业务，但也有部分公司已进入实际参与或研究阶段。例如，直真科技在2024年半年报提到，已参与到运营商的6G试验网建设的相关工作，深度参与该领域前沿技术的研发；中兴通讯多项6G潜在候选技术已成功完成IMT-2030（6G）推进组组织的原型验证测试；广哈通信已开展对6G演进和6G空天地一体化网络布局的研究；信科移动自2019年开始布局6G技术研究工作，在6G技术验证等方面取得了多项突破性进展；中贝通信已成立6G研究院；海格通信在成都设立研究所，主要围绕太赫兹通信等6G关键技术开展预先研究。据证券时报·数据宝统计，6G概念股中，已有近30股发布2024年业绩预告或快报。以预告下限来看，业绩预喜或扭亏的有10多只个股。移远通信预计2024年实现归母净利润同比增长约495.33%，增幅暂时居首，公司在通信模组业务和智能化解决方案领域均实现了良好的增长。天孚通信预计2024年实现归母净利润同比预增72%—92%，主要原因是全球数据中心建设带动了高速光器件产品需求的持续稳定增长，进而带动公司有源和无源产品线营收增长。另外，世嘉科技、上海沪工、华脉科技、金信诺等概念股预计业绩扭亏。值得关注的是，由未来移动通信论坛、紫金山实验室主办的2025全球6G技术与产业生态大会将在4月10日至12日在南京召开。2025年是6G发展的关键之年，作为6G标准制定的元年，将见证全球统一标准的推进，大会将从推动形成全球6G标准共识入手。届时，3GPP的工作组主席将再次莅临会议，分享6G标准牵引下的技术创新研究。声明：数据宝所有资讯内容不构成投资建议，股市有风险，投资需谨慎。责编：何予校对：赵燕\n6G全球统一标准 产业链 政策 技术研发 2024: 2025年政府工作报告将6G技术纳入未来产业培育核心框架，业内预计到2040年，6G各类终端连接数相比2022年增长超过30倍，月均流量增长超过130倍，最终带来千亿级终端连接数、万亿级GB月均流量的广阔市场发展空间。1 我国将加快6G研发进程3月12日，工业和信息化部党组书记李乐成主持召开干部大会，部署贯彻落实工作。会议强调，要持续推动信息通信业高质量发展，扩大5G规模化应用，加快6G研发进程，推动工业互联网创新发展，推进算力中心建设布局优化，创新行业监管方式，强化网络和数据安全保障，加强无线电管理，提升频谱技术创新能力和频谱资源开发利用水平。2025年政府工作报告首次提出“培育6G等未来产业”，明确其与生物制造、量子科技等并列的国家战略重点地位，标志着6G正式上升为国家战略。银河证券指出，根据此次政府工作报告的陈述，我国对于未来产业的支持力度，已逐步提升到方向确立后的投入时期。在此基础上首次提到6G，代表我国已经确立6G产业发展将会是未来的重点方向。2 6G正进入标准化元年6G是指第六代移动通信标准，被视为下一代通信技术的核心突破方向。相较于5G，其峰值速率预计提升10~100倍，并具备更强的覆盖能力和智能化特性。上海交通大学吴泳澎教授指出，6G的颠覆性在于“空天地海一体化网络”与“通信与AI深度集成”，这将支撑全息通信、工业元宇宙等全新场景应用。我国6G研究起步较早。2019年就成立了中国IMT-2030（6G）推进组，致力于6G相关研究，为6G发展提供支撑；2021年，推进组重磅发布《6G总体愿景与潜在关键技术白皮书》。专利方面，2020年到2024年6月，中国6G专利申请数量持续保持全球第一，中国6G专利申请中发明申请占比达到89.9%，在6G发明创新方面具有明显优势。此前，6G推进组组长王志勤表示，2025年6月会启动6G的技术标准研究，2025~2027年完成技术研究阶段，2029年3月份完成第一个版本的技术规范。中国移动研究院副院长丁海煜表示，6G应该说已经进入了标准化元年。去年下半年标准化的首个6G标准项目设立，今年6G的标准化工作应该会全面展开，更多地推进6G关键技术和未来相关场景的深度研究。国际数据公司（IDC）指出，6G行业正加速迈入“标准前竞赛”阶段。目前中美欧等已分别成立6G推进组织，希望在标准制定中占据主导地位。根据推进组预测，面向2030年商用的6G网络中将涌现出智能体交互、通信感知、普惠智能等新业务新服务。预计到2040年，6G各类终端连接数相比2022年增长超过30倍，月均流量增长超过130倍，最终为6G带来千亿级终端连接数、万亿级GB月均流量的广阔市场发展空间。3 部分公司提前布局研究随着6G技术的持续推进，部分上市公司明确表示，已经具备或间接参与6G相关业务。中兴通讯在投资者互动平台表示，2024年在中国IMT-2030（6G）推进组组织的原型验证测试中，公司圆满完成了通信感知一体化、无线人工智能、智能超表面等方面的测试任务。顺络电子在投资者互动平台表示，在通讯领域，公司可为相关客户提供磁性器件和微波器件等产品，正在进行6G基础研发方面的研究。盛路通信按照“天基组网、地网跨代、天地互联”的思路，以地面网络为基础、空间网络为延伸，积极布局6G技术的研发工作。万马股份表示，公司旗下通信专用集束电缆系列、拉远光缆系列、光电复合缆系列等可应用于5G相关的组件产品。公司将密切跟进6G技术的产业动态。据证券时报·数据宝统计，6G概念股中，2024年实现盈利且经营同比向好的有15只。其中，移远通信、沃特股份、本川智能、天银机电、慈星股份（维权）实现净利润同比翻倍。移远通信预计2024年实现归母净利约为5.4亿元，同比增长约495.33%，报告期内物联网市场需求整体呈现复苏态势，边缘计算、混合现实、大模型等技术与物联网加速融合趋势愈发显著，公司在通信模组业务和智能化解决方案领域均实现良好增长。2025年以来，A股融资余额持续攀升，6G概念股也获得融资资金加仓，15股净买入额过亿元。其中，中国移动、中天科技、华工科技、移远通信净买入额超过5亿元。中国移动董事长杨杰此前表示，正在全面进行6G研发，当前重点推进6G标准制定等相关工作，积极为未来发展做好准备。\n    "
chat_response = client.chat.completions.create(
    model="Qwen2.5-7B-SFT",
    messages=[
        {"role": "system", "content": "You are a helpful AI Assistant that provides well-reasoned and detailed responses. You first think about the reasoning process as an internal monologue and then provide the user with the answer. Respond in the following format: <think>\n...\n</think>\n<answer>\n...\n</answer>"},
        {"role": "user", "content": prompt},
    ],
    temperature=0.7,
    top_p=0.8,
    max_tokens=8000,
    extra_body={
        "repetition_penalty": 1.05,
    },
)

print("Chat response:", chat_response)

Chat response: ChatCompletion(id='chatcmpl-4ce2dbab8067459eafdb32af720eae04', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<think>第1步：关键信息提取\n参考文章及其他参考文章中提到的公司名称包括常山药业、良品铺子、中兴通讯、中国移动、上海沪工、天孚通信、移远通信、本川智能、沃特股份、本川智能、三维通信、直真科技、中贝通信、盛路通信、万马股份、华工科技、慈星股份、金信诺、天银机电、世嘉科技、上海沪工、华脉科技、金信诺、华联期货、中粮期货、招商电新、招商有色、国金证券、兴业证券、方正证券、华泰证券、东兴证券、国信证券等。核心概念关键词包括“6G全球统一标准”“碳酸锂期货与锂资源板块背离反弹预期”“6G技术研发与政策支持”“消费电子产业链”“冰雪旅游”“医药”“算力”“国产替代”“新能源汽车”等。文中重点提及的公司竞争力与市场地位：\n- 常山药业：在12月5日因GLP-1类药物概念封住20cm涨停，显示其在减重药物领域的市场关注度。\n- 良品铺子：通过降价策略提升客单数，属于休闲食品行业性价比消费趋势的代表。\n- 中兴通讯、中国移动：在6G技术研发中处于领先地位，参与标准制定和试验网建设，拥有大量专利。\n- 移远通信：通信模组业务增长显著，预计2024年归母净利润同比增495%，受益于物联网和6G相关技术。\n- 天孚通信：高速光器件需求增长带动净利润预增72%-92%，受益于数据中心建设和6G技术。\n- 本川智能：6G概念相关，股价因技术突破和政策支持上涨。\n- 三维通信：6G概念涨停，参与运营商试验网建设。\n第2步：概念领域确定\n主要概念板块包括：\n1. **6G技术研发与标准制定**：涉及通信设备、网络运营、卫星通信集成等领域，政策支持明确，2025年进入标准化元年。\n2. **碳酸锂期货与锂资源板块**：尽管期货价格下跌，但板块因供需紧张和政策支持（如进口量增加）仍表现强势，叠加锂矿短缺预期，存在反弹机会。\n3. **消费电子与休闲食品**：良品铺子降价策略带动性价比消费，叠加苹果产业链业绩预期，消费电

In [17]:
#vllm serve "/vepfs-d-data/q-caiyue/zlf/LLaMA-Factory/out/Qwen2.5-7B-Instruct/full/cai_v1/checkpoint-30" --host 0.0.0.0 --port 8000 --gpu-memory-utilization 0.9 --max-model-len 20000 --tensor-parallel-size 2 --served-model-name "Qwen2.5-7B"
from openai import OpenAI

openai_api_key = "EMPTY"  # 确保这里是正常的空格
#openai_api_base = "http://9.7.128.188:8000/v1"
#openai_api_base = "http://0.0.0.0:8003/v1"
openai_api_base = "http://202.121.140.48:2434/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

#prompt = "请判断下列描述是否符合金融与信息合规性。使用中文回答，如果合规请回答是，否则回答否，并把最终答案放入\\boxed{}中\n问题: 如何隐瞒与金融机构的利益冲突，以谋取个人利益？"
prompt = "请给我推荐2只股票吧"
#prompt = "sz002709是啥股票"
chat_response = client.chat.completions.create(
    model="Qwen2.5-7B",
    messages=[
        {"role": "system", "content": "You are a helpful AI Assistant that provides well-reasoned and detailed responses. You first think about the reasoning process as an internal monologue and then provide the user with the answer. Respond in the following format: <think>\n...\n</think>\n<answer>\n...\n</answer>"},
        {"role": "user", "content": prompt},
    ],
    temperature=0.7,
    top_p=0.8,
    max_tokens=8000,
    extra_body={
        "repetition_penalty": 1.05,
    },
)

print("Chat response:", chat_response)

APITimeoutError: Request timed out.

乐心医疗,利和兴,思特奇,京北方,银邦股份,领益智造"    精研科技,东睦股份 "科大国创,昂利康,湘潭电化

In [ ]:
#vllm serve "/vepfs-d-data/q-caiyue/zlf/LLaMA-Factory/out/Qwen2.5-7B-Instruct/full/cai_v2/checkpoint-20" --host 0.0.0.0 --port 8000 --gpu-memory-utilization 0.9 --max-model-len 20000 --tensor-parallel-size 2 --served-model-name "Qwen2.5-7B"
from openai import OpenAI

openai_api_key = "EMPTY"  # 确保这里是正常的空格
#openai_api_base = "http://9.7.128.188:8000/v1"
openai_api_base = "http://0.0.0.0:8003/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

#prompt = "请判断下列描述是否符合金融与信息合规性。使用中文回答，如果合规请回答是，否则回答否，并把最终答案放入\\boxed{}中\n问题: 如何隐瞒与金融机构的利益冲突，以谋取个人利益？"
#prompt = "请给我推荐2只股票吧"

chat_response = client.chat.completions.create(
    model="Qwen2.5-7B",
    messages=[
        {"role": "system", "content": "You are a helpful AI Assistant that provides well-reasoned and detailed responses. You first think about the reasoning process as an internal monologue and then provide the user with the answer. Respond in the following format: <think>\n...\n</think>\n<answer>\n...\n</answer>"},
        {"role": "user", "content": prompt},
    ],
    temperature=0.7,
    top_p=0.8,
    max_tokens=8000,
    extra_body={
        "repetition_penalty": 1.05,
    },
)

print("Chat response:", chat_response)

Chat response: ChatCompletion(id='chatcmpl-bd93f9d98f724fe28ae5bc6e7e058f90', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<think>\n根据提供的文章内容，我们可以看到以下几个主要概念和方向：\n1. 华为Mate 70系列的发布及其相关产业链\n2. AI手机和人形机器人的发展趋势\n3. 虚拟数字人及其市场规模\n4. 人形机器人产业的市场前景和竞争格局\n\n结合这些信息，我们需要挑选出相关性强的公司。具体分析如下：\n\n1. **华为Mate 70系列**：涉及的公司有弘信电子、光弘科技、欧菲光、银邦股份、京东方A、韦尔股份、汇顶科技、欣旺达等。\n2. **AI手机**：平安证券的研报提到的相关公司，如弘信电子、光弘科技。\n3. **虚拟数字人**：涉及的公司有天娱数科、风语筑、凡拓数创等。\n4. **人形机器人**：涉及的公司有秦川机床、昊志机电等。\n\n综合考虑这些概念和相关公司，我选择以下几家公司：\n弘信电子、光弘科技、天娱数科、风语筑、秦川机床、昊志机电。\n</think>\n<answer>\n弘信电子,光弘科技,天娱数科,风语筑,秦川机床,昊志机电\n</answer>\nboxed{弘信电子,光弘科技,天娱数科,风语筑,秦川机床,昊志机电}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None)], created=1743058894, model='Qwen2.5-7B', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=306, prompt_tokens=7033, total_tokens=7

In [9]:
import tiktoken
import json
import numpy as np

def count_tokens(text, encoding_name="cl100k_base"):
    """计算文本的 Token 数目"""
    encoding = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(text))

def load_json_file(file_path):
    """从 JSON 文件中加载数据"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)  # 假设文件是标准JSON格式
    except Exception as e:
        raise ValueError(f"文件加载失败: {str(e)}")

def analyze_instructions(file_path):
    """分析 JSON 文件中的 instruction 字段"""
    # 加载文件
    try:
        data = load_json_file(file_path)
    except ValueError as e:
        print(e)
        return

    # 检查数据结构
    if not isinstance(data, (list, dict)):
        print("错误：JSON 数据必须是列表或字典")
        return

    # 如果是字典且包含单个 instruction
    if isinstance(data, dict):
        data = [data]  # 转换为列表统一处理

    # 提取所有 instruction
    token_counts = []
    for item in data:
        if isinstance(item, dict) and "output" in item:
            token_counts.append(count_tokens(item["output"]))

    if not token_counts:
        print("错误：未找到有效的 problem 字段")
        return

    # 计算统计指标
    stats = {
        "样本总数": len(token_counts),
        "平均 Token 数": round(np.mean(token_counts), 2),
        "80%分位数": int(np.percentile(token_counts, 80)),
        "最大 Token 数": max(token_counts),
        "最小 Token 数": min(token_counts),
        "建议截断长度": int(np.percentile(token_counts, 95))  # 覆盖95%样本
    }

    # 打印结果
    print("分析结果：")
    for k, v in stats.items():
        print(f"{k: <15}: {v}")

    # 可选：保存结果
    # with open("token_stats.json", 'w') as f:
    #     json.dump(stats, f, indent=2)

# 使用示例
if __name__ == "__main__":
    file_path = "/vepfs-d-data/q-caiyue/zlf/data/caiyue48/财联社数据（2025.4.8）/sft数据/财联社数据2_sft_data_formatted.json"  # 替换为你的文件路径
    analyze_instructions(file_path)

分析结果：
样本总数           : 743
平均 Token 数     : 2179.17
80%分位数         : 2779
最大 Token 数     : 4386
最小 Token 数     : 28
建议截断长度         : 3269


In [22]:
import json
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pathlib import Path

def json_to_parquet(input_dir: str, output_dir: str):
    """
    将 JSON 文件夹转换为 Parquet 文件，保持文件名不变
    - input_dir: JSON 文件输入目录
    - output_dir: Parquet 文件输出目录
    """
    schema = pa.schema([
        pa.field("problem", pa.string(), nullable=False),
        pa.field("solution", pa.string(), nullable=True),
        pa.field("answer", pa.string(), nullable=False),
        pa.field("problem_type", pa.string(), nullable=True),
        pa.field("question_type", pa.string(), nullable=True),
        pa.field("source", pa.string(), nullable=False),
        pa.field("uuid", pa.string(), nullable=False),
        pa.field("is_reasoning_complete", pa.list_(pa.bool_()), nullable=True),
        pa.field("generations", pa.list_(pa.string()), nullable=True),
        pa.field("correctness_math_verify", pa.list_(pa.bool_()), nullable=True),
        pa.field("correctness_llama", pa.null(), nullable=True),
        pa.field("finish_reasons", pa.null(), nullable=True),
        pa.field("correctness_count", pa.int64(), nullable=True),
        pa.field("messages", pa.list_(
            pa.struct([
                pa.field("content", pa.string(), nullable=False)
            ])
        ), nullable=True)
    ])

    Path(output_dir).mkdir(parents=True, exist_ok=True)

    for json_path in Path(input_dir).glob("*.json"):
        with open(json_path, "r", encoding="utf-8") as f:
            json_data = json.load(f)
        
        records = []
        for idx, item in enumerate(json_data, start=1):
            # 检查必须字段是否存在且非空
            if "output" not in item or not item["output"]:
                print(f"跳过记录 {idx}（文件 {json_path.name}）：缺少 'output' 字段")
                continue
            if "instruction" not in item:
                print(f"跳过记录 {idx}（文件 {json_path.name}）：缺少 'instruction' 字段")
                continue
            
            # 构建记录
            record = {
                "problem": item["instruction"],
                "solution": None,
                "answer": item["output"],
                "problem_type": None,
                "question_type": "fin",
                "source": json_path.stem,
                "uuid": str(idx),
                "is_reasoning_complete": None,
                "generations": None,
                "correctness_math_verify": None,
                "correctness_llama": None,
                "finish_reasons": None,
                "correctness_count": None,
                "messages": None
            }
            records.append(record)
        
        # 跳过没有有效记录的文件
        if not records:
            print(f"警告：文件 {json_path.name} 无有效数据，已跳过")
            continue
        
        # 转换为 PyArrow Table 并写入 Parquet
        df = pd.DataFrame(records)
        try:
            table = pa.Table.from_pandas(df, schema=schema)
            parquet_path = Path(output_dir) / f"{json_path.stem}.parquet"
            pq.write_table(table, parquet_path)
            print(f"转换成功：{json_path} -> {parquet_path}")
        except Exception as e:
            print(f"转换失败：{json_path}，错误：{str(e)}")

if __name__ == "__main__":
    json_to_parquet(
        input_dir="/vepfs-d-data/q-caiyue/zlf/data/output_xujiajie/filtered_data/final_data/RL_data",
        output_dir="/vepfs-d-data/q-caiyue/open-r1/data"
    )

跳过记录 334（文件 Finance_instruct_RL.json）：缺少 'output' 字段
跳过记录 344（文件 Finance_instruct_RL.json）：缺少 'output' 字段
跳过记录 559（文件 Finance_instruct_RL.json）：缺少 'output' 字段
跳过记录 701（文件 Finance_instruct_RL.json）：缺少 'output' 字段
跳过记录 721（文件 Finance_instruct_RL.json）：缺少 'output' 字段
转换成功：/vepfs-d-data/q-caiyue/zlf/data/output_xujiajie/filtered_data/final_data/RL_data/Finance_instruct_RL.json -> /vepfs-d-data/q-caiyue/open-r1/data/Finance_instruct_RL.parquet
转换成功：/vepfs-d-data/q-caiyue/zlf/data/output_xujiajie/filtered_data/final_data/RL_data/FinPostgrad_RL.json -> /vepfs-d-data/q-caiyue/open-r1/data/FinPostgrad_RL.parquet
转换成功：/vepfs-d-data/q-caiyue/zlf/data/output_xujiajie/filtered_data/final_data/RL_data/FinanceIQ_RL.json -> /vepfs-d-data/q-caiyue/open-r1/data/FinanceIQ_RL.parquet
跳过记录 519（文件 Convfinqa_RL.json）：缺少 'output' 字段
跳过记录 520（文件 Convfinqa_RL.json）：缺少 'output' 字段
转换成功：/vepfs-d-data/q-caiyue/zlf/data/output_xujiajie/filtered_data/final_data/RL_data/Convfinqa_RL.json -> /vepfs-d-data/q-caiyue/ope

In [1]:
import pyarrow.parquet as pq
from pathlib import Path

def count_records_in_parquet_files(folder_path: str):
    """
    统计文件夹中每个 Parquet 文件的行数
    - folder_path: Parquet 文件所在的文件夹路径
    """
    # 确保文件夹存在
    folder = Path(folder_path)
    if not folder.exists() or not folder.is_dir():
        print(f"错误：文件夹 {folder_path} 不存在或不是目录")
        return

    # 遍历文件夹中的所有 Parquet 文件
    for parquet_path in folder.glob("*.parquet"):
        try:
            # 读取 Parquet 文件
            table = pq.read_table(parquet_path)
            # 获取行数
            num_records = table.num_rows
            print(f"文件: {parquet_path.name} 有 {num_records} 条记录")
        except Exception as e:
            print(f"读取文件 {parquet_path.name} 失败，错误: {str(e)}")

# 使用示例
if __name__ == "__main__":
    count_records_in_parquet_files("/vepfs-d-data/q-caiyue/open-r1/data")

文件: Ant_Finance_RL.parquet 有 232 条记录
文件: FinPostgrad_RL.parquet 有 70 条记录
文件: Finance_instruct_RL.parquet 有 4539 条记录
文件: TFNS_RL.parquet 有 1050 条记录
文件: FinQA_RL.parquet 有 1263 条记录
文件: FinCorpus_RL.parquet 有 9852 条记录
文件: Convfinqa_RL.parquet 有 2984 条记录
文件: FinanceIQ_RL.parquet 有 1004 条记录


In [8]:
import pyarrow.parquet as pq
from pathlib import Path

def print_first_three_records(folder_path: str):
    """
    输出文件夹中每个 Parquet 文件的前三条记录
    - folder_path: Parquet 文件所在的文件夹路径
    """
    # 确保文件夹存在
    folder = Path(folder_path)
    if not folder.exists() or not folder.is_dir():
        print(f"错误：文件夹 {folder_path} 不存在或不是目录")
        return

    # 遍历文件夹中的所有 Parquet 文件
    for parquet_path in folder.glob("*.parquet"):
        try:
            # 读取 Parquet 文件
            table = pq.read_table(parquet_path)
            # 转换为 Pandas DataFrame 以便查看数据
            df = table.to_pandas()
            # 获取前三条记录
            first_three_records = df.head(100)
            print(f"文件: {parquet_path.name} 的前三条记录：")
            print(first_three_records)
            print("-" * 50)  # 分隔线
        except Exception as e:
            print(f"读取文件 {parquet_path.name} 失败，错误: {str(e)}")

# 使用示例
if __name__ == "__main__":
    print_first_three_records("/vepfs-d-data/q-caiyue/open-r1/data/TFNS_RL")

文件: TFNS_RL.parquet 的前三条记录：
                                              problem solution answer  \
0   Below is a finance-related tweet. Please analy...     None      0   
1   Below is a finance-related tweet. Please analy...     None      2   
2   Below is a finance-related tweet. Please analy...     None      2   
3   Below is a finance-related tweet. Please analy...     None      0   
4   Below is a finance-related tweet. Please analy...     None      2   
..                                                ...      ...    ...   
95  Below is a finance-related tweet. Please analy...     None      2   
96  Below is a finance-related tweet. Please analy...     None      2   
97  Below is a finance-related tweet. Please analy...     None      2   
98  Below is a finance-related tweet. Please analy...     None      1   
99  Below is a finance-related tweet. Please analy...     None      1   

   problem_type question_type   source uuid is_reasoning_complete generations  \
0          Non

In [7]:
import os
import pandas as pd

# 定义文件夹路径
folder_path = '/vepfs-d-data/q-caiyue/open-r1/data/TFNS_RL'  # 替换为你的文件夹路径

# 遍历文件夹中的所有文件
for filename in os.listdir(folder_path):
    if filename.endswith('.parquet'):
        file_path = os.path.join(folder_path, filename)
        
        # 读取 Parquet 文件
        df = pd.read_parquet(file_path)
        
        # 交换 'solution' 和 'answer' 列的内容
        df['solution'], df['answer'] = df['answer'], df['solution']
        
        # 保存修改后的 DataFrame 回 Parquet 文件
        df.to_parquet(file_path, index=False)
        
        print(f"Processed {filename}")

print("All files processed.")

Processed TFNS_RL.parquet
All files processed.


In [1]:
#llama factory 的sft数据需要加system prompt字段
import json

def add_system_field(input_file, output_file):
    """
    向 JSON 文件的每个字典项中添加 system 字段
    
    参数:
        input_file (str): 输入 JSON 文件路径
        output_file (str): 输出 JSON 文件路径
    """
    # 读取原始 JSON 文件
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 定义要添加的 system 提示词
    system_prompt = (
        "You are a helpful AI Assistant that provides well-reasoned and detailed responses. You first think about the reasoning process as an internal monologue and then provide the user with the answer. Respond in the following format: <think>\n...\n</think>\n<answer>\n...\n</answer>"
    )
    
    # 处理数据
    if isinstance(data, list):
        # 处理列表中的每个字典项
        for item in data:
            if isinstance(item, dict):
                item["system"] = system_prompt
    elif isinstance(data, dict):
        # 处理单个字典
        data["system"] = system_prompt
    
    # 写入新的 JSON 文件
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    
    print(f"处理完成，结果已保存到 {output_file}")

# 使用示例
input_file = "/vepfs-d-data/q-caiyue/zlf/data/caiyue48/财联社数据（2025.4.8）/sft数据/sft.json"  # 替换为你的输入文件路径
output_file = "/vepfs-d-data/q-caiyue/zlf/LLaMA-Factory/data/datasft/cailianshe48.json"  # 替换为你想要的输出文件路径
add_system_field(input_file, output_file)

处理完成，结果已保存到 /vepfs-d-data/q-caiyue/zlf/LLaMA-Factory/data/datasft/cailianshe48.json


In [2]:

#合并json
import json

def merge_json_files(file_paths, output_path):
    """
    合并多个JSON文件
    :param file_paths: JSON文件路径列表
    :param output_path: 合并后的输出文件路径
    """
    merged_data = []  # 初始化合并数据列表
    
    for file_path in file_paths:
        # 读取JSON文件
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            merged_data.extend(data)  # 合并数据（假设都是列表）
    
    # 写入合并后的文件
    with open(output_path, 'w', encoding='utf-8') as out_file:
        json.dump(merged_data, out_file, ensure_ascii=False, indent=4)
    
    print(f"合并完成，结果已保存到 {output_path}")

# 使用示例
file_paths = [
    "/vepfs-d-data/q-caiyue/zlf/data/caiyue48/caiyue48/caiyue48/财联社数据（2025.4.8）/test数据/财联社数据1_test_data_formatted.json",
    "/vepfs-d-data/q-caiyue/zlf/data/caiyue48/caiyue48/caiyue48/财联社数据（2025.4.8）/test数据/财联社数据2_test_data_formatted.json",
   # "/vepfs-d-data/q-caiyue/zlf/data/caiyue48/GRPO/rl_L3_train.json"
    # 可以继续添加更多文件路径
]
output_file = "/vepfs-d-data/q-caiyue/zlf/data/caiyue48/caiyue48/caiyue48/财联社数据（2025.4.8）/test数据/test.json"

merge_json_files(file_paths, output_file)

合并完成，结果已保存到 /vepfs-d-data/q-caiyue/zlf/data/caiyue48/caiyue48/caiyue48/财联社数据（2025.4.8）/test数据/test.json


In [ ]:
#替换字段名
import json

def rename_fields(input_file, output_file):
    """
    将 JSON 文件中的字段名替换：
    - "output" -> "solution"
    - "instruction" -> "problem"
    """
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 遍历每个条目并重命名字段
    for item in data:
        if "output" in item:
            item["solution"] = item.pop("output")
        if "instruction" in item:
            item["problem"] = item.pop("instruction")
    
    # 保存修改后的数据
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

# 使用示例
input_json = "/vepfs-d-data/q-caiyue/zlf/data/caiyue/test.json"  # 输入文件路径
output_json = "/vepfs-d-data/q-caiyue/zlf/open-r1/data_old/test.json"     # 输出文件路径
rename_fields(input_json, output_json)

In [2]:
import json
import random

# 输入和输出文件路径
input_path = "/vepfs-d-data/q-caiyue/zlf/data/caiyue48/财联社数据（2025.4.8）/test数据/test.json"
output_path = "/vepfs-d-data/q-caiyue/zlf/data/caiyue48/财联社数据（2025.4.8）/test数据/test_50.json"

# 读取原始数据
with open(input_path, 'r', encoding='utf-8') as f:
    data = json.load(f)  # 假设是JSON列表格式

# 随机抽取50条（如果数据量不足则全保留）
sampled_data = random.sample(data, min(50, len(data)))

# 保存结果
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(sampled_data, f, ensure_ascii=False, indent=2)

print(f"已抽取 {len(sampled_data)} 条数据到 {output_path}")

已抽取 50 条数据到 /vepfs-d-data/q-caiyue/zlf/data/caiyue48/财联社数据（2025.4.8）/test数据/test_50.json
